# 4 Goal

### using qraph aql and a medicine name find an alternative maedicine with a lower cost

# By - Reva Bharara


In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from arango import ArangoClient

In [4]:
# creating the database connection
client=ArangoClient(hosts="http://localhost:8529/")
med_db=client.db(name="drug_repurposing_3" , username="root", password="")

# ==============================================================
# Alternative Medicine queries
# ==============================================================

In [28]:
# getting input medicine of which we need alternate medicine

medicine_names = ["Augmentin 625 Duo Tablet", "Aricep 5 Tablet", "Avil 25 Tablet","Azithral 500 Tablet","Aciloc 150 Tablet","Ascoril LS Syrup","Allegra 120mg Tablet"]

In [39]:
#getting the medicine id of the input medicine

original_prices = []
for med in medicine_names:
    cursor=med_db.aql.execute(f'for med in Medicine filter med.name=="{med}" return med.MRP')
    for price in cursor:
        original_prices.append(price)

print(medicine_names)
print(original_prices)

['Augmentin 625 Duo Tablet', 'Aricep 5 Tablet', 'Avil 25 Tablet', 'Azithral 500 Tablet', 'Aciloc 150 Tablet', 'Ascoril LS Syrup', 'Allegra 120mg Tablet']
[452, 390, 347, 105, 94, 354, 46]


In [41]:
ids = []
for med in medicine_names:
    cursor=med_db.aql.execute(f'for med in Medicine filter med.name=="{med}" return med._id')
    for id in cursor:
        ids.append(id)

ids

['Medicine/2157707',
 'Medicine/2157709',
 'Medicine/2157711',
 'Medicine/2157713',
 'Medicine/2157715',
 'Medicine/2157717',
 'Medicine/2157719']

In [7]:
medicine_id=str
for med_id in cursor:
    medicine_id=med_id

In [42]:
ids

['Medicine/2157707',
 'Medicine/2157709',
 'Medicine/2157711',
 'Medicine/2157713',
 'Medicine/2157715',
 'Medicine/2157717',
 'Medicine/2157719']

In [ ]:
# getting the chemical composition of the input medicine and representing it in a dataframe
cursor=med_db.aql.execute(f'for v,e in 1..1 outbound "{medicine_id}" graph medical_kg filter IS_SAME_COLLECTION("has_chemical",e) return {{"chemical":v._id, "concentration": e.weight}}')
med_composition=pd.DataFrame(cursor)

In [48]:
med_composition

""


In [13]:
# here we are going from the chemicals to other medicines that have the same concentration of those chemicals
# using the for loop for accessing every medicine for each individual chemical with the same composition as in the original medicine
chem_dataframes={}
for i in range(len(med_composition)):
    cursor=med_db.aql.execute(f'for v,e in 1..1 inbound "{med_composition.iloc[i,0]}" graph drug_kg_approach_3_ filter IS_SAME_COLLECTION("has_chemical",e) and e.weight=="{med_composition.iloc[i,1]}" return {{"medicine":v._id, "concentration": e.weight}}')
    chem_dataframes[f"chem{i+1}"]=pd.DataFrame(cursor)

In [14]:
# now we will be creting an intersection of sets of different medicines so that we get the prospected alternate medicine
intersection = set(chem_dataframes['chem1'].medicine)
for chem_df in chem_dataframes.values():
    intersection=intersection&set(chem_df.medicine)

In [ ]:
intersection

In [ ]:
# now we need to shortlist the medicines such that only those particular salts are contained and not more than that
for i in intersection:
    cursor=med_db.aql.execute(f'let data = (for v,e in 1..1 outbound "{i}" graph drug_kg_approach_3_ filter IS_SAME_COLLECTION("has_chemical",e) return {{"chemical":v._id, "concentration": e.weight}})return length(data)')
    for j in cursor:
        print(i,j)
        
# as in the medicine above there are only two salts so we need to shortlist only the medicines that have two salts not more than that

In [19]:
alt_medicine=[]
for i in intersection:
    cursor=med_db.aql.execute(f'let data = (for v,e in 1..1 outbound "{i}" graph drug_kg_approach_3_ filter IS_SAME_COLLECTION("has_chemical",e) return {{"chemical":v._id, "concentration": e.weight}})return length(data)')
    for j in cursor:
        if j==len(med_composition):
            alt_medicine.append(i)

In [20]:
# getting only alternate medicines that have MRP less than the entered og medicine
alt_med_lst=list()
for medicine in alt_medicine:
    cursor=med_db.aql.execute(f"return document('{medicine}')")
    for i in cursor:
        alt_med_lst.append(i)
    
alt_med_price_df=pd.DataFrame(alt_med_lst)   
alt_med_price_df.drop(columns=['_key','_rev'], inplace=True)
alt_med_price_df.set_index('_id', inplace=True)
alt_med_price_df.head()

,name,MRP,Habit Forming
_id,,,
Medicine/2159195,Aginal 5 Tablet,138,No
Medicine/2227227,Yewloc 5mg Tablet,384,No
Medicine/2158657,Amodep 5 Tablet,315,No
Medicine/2157997,Amlodac 5 Tablet,428,No
Medicine/2197391,Numlo 5mg Tablet,298,No


In [21]:
filter_price=alt_med_price_df['MRP']<alt_med_price_df.loc[medicine_id,'MRP']
alt_med_price_df=alt_med_price_df.loc[filter_price]

In [22]:
alt_med_price_df

,name,MRP,Habit Forming
_id,,,
Medicine/2159195,Aginal 5 Tablet,138,No
Medicine/2197391,Numlo 5mg Tablet,298,No
Medicine/2186965,Klide 5mg Tablet,11,No
Medicine/2158523,Amlogard 5mg Tablet,127,No
Medicine/2168915,Danavish Amlodipine 5mg Tablet,297,No
Medicine/2158141,Amtas 5 Tablet,268,No
Medicine/2190199,Lupidip 5 Tablet,40,No
Medicine/2165377,Calchek 5 Tablet,263,No
Medicine/2161047,Biodipin 5mg Tablet,24,No


In [ ]:
alt_medicine

In [24]:
# now we need to get all the medicine names instead of their corresponding ids
alt_med_name=[]
for med in alt_medicine:
    cursor=med_db.aql.execute(f"return document('{med}').name")
    for m in cursor:

        alt_med_name.append(m)
    

In [25]:
alt_med_name 
# ALTERNATE MEDICINE NAMES

['Aginal 5 Tablet',
 'Yewloc 5mg Tablet',
 'Amodep 5 Tablet',
 'Amlodac 5 Tablet',
 'Numlo 5mg Tablet',
 'Amcard 5 Tablet',
 'Espin 5 Tablet',
 'Wellamo 5 Tablet',
 'Klide 5mg Tablet',
 'Amlopres 5 Tablet',
 'Amlip 5 Tablet',
 'Milopin 5mg Tablet',
 'Amlopin 5 Tablet',
 'Amlogard 5mg Tablet',
 'Danavish Amlodipine 5mg Tablet',
 'Amtas 5 Tablet',
 'Amlo 5mg Tablet',
 'Lupidip 5 Tablet',
 'Primodil 5 Tablet',
 'Calchek 5 Tablet',
 'Biodipin 5mg Tablet',
 'Amlokind 5 Tablet',
 'Amlovas 5 Tablet',
 'Stamlo 5 Tablet',
 'Elpress 5mg Tablet']

In [26]:
alt_med_name_less_price=alt_med_price_df[['name',"MRP"]].sort_values("MRP")

In [27]:
alt_med_name_less_price.iloc[0:3]
# ALTENATE MEDICINES WITH LESS PRICE

,name,MRP
_id,,
Medicine/2186965,Klide 5mg Tablet,11
Medicine/2161047,Biodipin 5mg Tablet,24
Medicine/2190199,Lupidip 5 Tablet,40


By - Aryan Rathore, Reva Bharara